In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon May 30 10:11:44 2022

@author: roy.369
"""


import numpy as np
import pickle

import matplotlib.pyplot as plt
import os
import sys
from scipy.interpolate import griddata
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.colorbar
from matplotlib import rc
mpl.rcParams['axes.linewidth'] = 3
import matplotlib as mpl  
mpl.rc('font',family='Times New Roman')
from matplotlib.pyplot import cm
from numpy.polynomial.polynomial import polyfit
from scipy.optimize import curve_fit
from matplotlib.ticker import ScalarFormatter
from matplotlib.ticker import LogLocator, NullFormatter

In [91]:
def arr_loc(val,X):
   ind = 0
   for i in range(len(X)):
      if(round(X[i],2) == round(val,2)):
         ind = i
         break
   return ind


def power_fit(x,a1,m):
    return a1*x**m
  
def linear_fit(x,a,b):
    return a+b*x
    
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx],idx

def func_fit(x,a,b):
    return a+b/(x**2)#+c/(x**4)

In [92]:
def r_point_upper_half_grid(N):

    r_lab = []
    r_rel_x = []
    r_rel_y = []
    x_span = int((int(N))/2)+1
    r_pair_1 = 0
    for i in range(x_span):
        for j in range(x_span):
            if(i<=j):
              r_lab.append(r_pair_1)  
              r_rel_x.append(i)
              r_rel_y.append(j)
              r_pair_1 = r_pair_1+1

    R_relative_1 = np.stack((r_rel_x,r_rel_y),axis = 1)
    return R_relative_1,r_pair_1

In [93]:
def r_point_full_grid(N):

    r_lab = []
    r_rel_x = []
    r_rel_y = []
    x_span = int((int(N))/2)+1
    r_pair_2 = 0
    for i in range(x_span):
        for j in range(x_span):
              r_lab.append(r_pair_2)
              r_rel_x.append(i)
              r_rel_y.append(j)
              r_pair_2 = r_pair_2+1

    R_relative_2 = np.stack((r_rel_x,r_rel_y),axis = 1)
    return R_relative_2,r_pair_2

In [94]:
def k_point_grid_upper_half_bz(N):

    K_lab = []
    Kx = []
    Ky = []
    x_span = int((int(N))/2)+1
    k_pair = 0
    for i in range(x_span):
        for j in range(x_span):
            if(i<=j):
              K_lab.append(k_pair)  
              Kx.append(i)
              Ky.append(j)
              k_pair = k_pair+1
    BZ = np.stack((K_lab,Kx,Ky),axis = 1)
    
    return BZ,k_pair


def k_point_grid_full_bz(N):

    K_lab = []
    Kx = []
    Ky = []
    x_span = int((int(N))/2)+1
    k_pair = 0
    for i in range(x_span):
        for j in range(x_span):
              K_lab.append(k_pair)
              Kx.append(i)
              Ky.append(j)
              k_pair = k_pair+1
    BZ = np.stack((K_lab,Kx,Ky),axis = 1)
    
    return BZ,k_pair


In [95]:
def adj_mat_calc(N):

    r_grid, r_rel_grid_size = r_point_upper_half_grid(int(N))
    r_rel_x = r_grid[:,0]
    r_rel_y = r_grid[:,1]

    r_2 = np.add(np.power(r_rel_x,2),np.power(r_rel_y,2))
    r_2_unq = np.sort(np.unique(r_2))

    rx_cord = []
    ry_cord = []
    r2 = []
    rx = np.arange(int(int(N)/2),-1*int(int(N)/2),-1)
    ry = np.arange(int(int(N)/2),-1*int(int(N)/2),-1)
    for i in range(len(rx)):
        for j in range(len(ry)):
            rx_cord.append(rx[i])
            ry_cord.append(ry[j])
            r2.append(rx[i]*rx[i]+ry[j]*ry[j])

    #plt.figure()
    #plt.scatter(rx_cord,ry_cord,c=r2)

    adj_mat = np.zeros(len(r_2_unq))
    for i in range(len(r_2_unq)):
        count = 0
        for i2 in range(len(r2)):
            if round(r2[i2],2) == round(r_2_unq[i],2):
               count = count+1
        adj_mat[i] = count
    
    return np.stack((r_2_unq,np.asarray(adj_mat)),axis = 1)  

In [117]:
def plot_func_multiple_beta_hf(N,u_val,mu,dtau,trot,plot_title,plot_x_label,plot_y_label,x_val,y_val,y_val_err,plot_name):

   color_1 = iter(["red","orange","goldenrod","blue","purple"]) #cm.gnuplot(np.linspace(0, 1, len(trot)+1)))
   plt.figure(figsize = (25,20))
   #plt.title(plot_title,fontsize = 60)
   plt.xticks([0,2,4,6,8,10,12]) #,fontsize = 120)
   plt.yticks([0,4,8,12]) #,fontsize = 120)
   plt.xlim(0,12) 
   plt.ylim(0,16) 
   #plt.xscale('log')
   #plt.yscale('log') 
   #plt.ylabel(plot_y_label,fontsize = 60)
   #plt.xlabel(plot_x_label,fontsize = 60)
   #U_val = np.linspace(3.00,12,num=101)
   #Hd_fit = np.zeros(len(U_val))
   plt.tick_params(axis='both', which='major', direction='out', pad=20,length = 30, width = 2.5,labelsize = 120)
   plt.tick_params(axis='both', which='minor', direction='out', pad=20,length = 15, width = 1.5,labelsize = 60)
   #for i in range(len(U_val)):
   #    Hd_fit[i] = 1/(U_val[i]**2)-1/(U_val[i]**4)
   for jj in range(len(trot)):
       c_1 = next(color_1)
       beta = (float(dtau))*(float(trot[jj]))
       T = round(1/beta,2)
       plt.errorbar(x_val,y_val[:,jj]*10**4,yerr = y_val_err[:,jj]*10**4,c=c_1,marker = 'o',markersize = 25, linewidth=2,elinewidth = 3, capsize = 5,label = r"T = %s"%str(T))

   #plt.plot(U_val,Hd_fit,linestyle = 'dashed',color=  'black',linewidth = 3)
   #plt.ylim(top=0.1)  
   popt,pcov = curve_fit(func_fit,x_val[-8:],y_val[-8:,-1])
   print(popt)
   #plt.plot(U_val, func_fit(U_val, *popt), c='black', linestyle = 'dashed',linewidth=5)
   #print(x_val[-8:], func_fit(x_val[-8:], *popt)) 

   #plt.gca().yaxis.set_minor_formatter(NullFormatter())
   plt.grid(True,which='both')
   plt.axvspan(3.1, 4.3, facecolor='r', alpha=0.2)
   #plt.axvspan(3.5, 4.3, facecolor='r', alpha=0.2)
   #plt.axvspan(4, 6, facecolor='b', alpha=0.2)

   #plt.legend(loc='best',fontsize = 80) 
   plt.tight_layout() 
   plt.savefig(plot_name)
   plt.close()

In [118]:
def equal_time_holon_doublon_correlation_function_half_filling(Text_dir_main_corr_hf,Text_dir_main_eqm,Graph_dir_hf,N,u,mu,dtau,L):

   r_grid, r_rel_grid_size = r_point_upper_half_grid(N)
   r_rel_x = r_grid[:,0]
   r_rel_y = r_grid[:,1]

   r_2 = np.add(np.power(r_rel_x,2),np.power(r_rel_y,2))
   r_2_unq = np.sort(np.unique(r_2))

   Dbn_Dbn_corr_hf = np.zeros((len(u),len(L),len(r_2_unq)))
   Dbn_Dbn_corr_hf_std = np.zeros((len(u),len(L),len(r_2_unq)))
   
   Den_Dbn_corr_hf = np.zeros((len(u),len(L),len(r_2_unq)))
   Den_Dbn_corr_hf_std = np.zeros((len(u),len(L),len(r_2_unq)))
   
   Hln_Dbn_corr_hf = np.zeros((len(u),len(L),len(r_2_unq)))
   Hln_Dbn_corr_hf_std = np.zeros((len(u),len(L),len(r_2_unq)))
 

   Dbn_Dbn_conn_corr_hf = np.zeros((len(u),len(L),len(r_2_unq)))
   Dbn_Dbn_conn_corr_hf_std = np.zeros((len(u),len(L),len(r_2_unq)))
   
   Den_Dbn_conn_corr_hf = np.zeros((len(u),len(L),len(r_2_unq)))
   Den_Dbn_conn_corr_hf_std = np.zeros((len(u),len(L),len(r_2_unq)))
   
   Hln_Dbn_conn_corr_hf = np.zeros((len(u),len(L),len(r_2_unq)))
   Hln_Dbn_conn_corr_hf_std = np.zeros((len(u),len(L),len(r_2_unq)))
   
   Nrm_Hln_Dbn_conn_corr_hf = np.zeros((len(u),len(L),len(r_2_unq)))
   Nrm_Hln_Dbn_conn_corr_hf_std = np.zeros((len(u),len(L),len(r_2_unq)))

   
   Nden = np.zeros((len(u),len(L)))
   Nden_std = np.zeros((len(u),len(L)))
   Ndbn = np.zeros((len(u),len(L)))
   Ndbn_std = np.zeros((len(u),len(L)))
   Nhln = np.zeros((len(u),len(L)))
   Nhln_std = np.zeros((len(u),len(L)))
    
   Hl_db = np.zeros((len(u),len(L)))
   Hl_db_std = np.zeros((len(u),len(L)))

    
   U_val = np.zeros(len(u))
   for i in range(len(u)):
       U_val[i] = float(u[i])
       
       
   for i in range(len(u)):
       for k in range(len(L)):
                       
           Text_dir_eqm = "%s/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Thermodynamic_measurements"%(Text_dir_main_eqm,N,u[i],dtau,mu,dtau,L[k])

           filename_eqm_avg = '%s/Thermodynamic_measurements_normal_averaged_dictionary_N_%s_U_%s_mu_%s_dtau_%s_L_%s.pkl' %(Text_dir_eqm,N,u[i],mu,dtau,L[k])
           with open(filename_eqm_avg, 'rb') as infile:
                sys_measure_avg = pickle.load(infile)

           filename_eqm_std = '%s/Thermodynamic_measurements_normal_standard_deviation_dictionary_N_%s_U_%s_mu_%s_dtau_%s_L_%s.pkl' %(Text_dir_eqm,N,u[i],mu,dtau,L[k])
           with open(filename_eqm_std, 'rb') as infile:
                sys_measure_std = pickle.load(infile)

           Nden[i][k] = sys_measure_avg['Density averaged']
           Nden_std[i][k] = sys_measure_std['Density standard deviation']
           Ndbn[i,k] = sys_measure_avg['Doublon number averaged']
           Ndbn_std[i,k] = sys_measure_std['Doublon number standard deviation']
           Nhln[i][k] = 1-Nden[i][k]+Ndbn[i][k]
           Nhln_std[i][k] = np.sqrt(Ndbn_std[i][k]*Ndbn_std[i][k]+Nden_std[i][k]*Nden_std[i][k])
           Hl_db[i][k] = Ndbn[i][k]*Nhln[i][k]
           Hl_db_std[i][k] = Hl_db[i][k]*(Ndbn_std[i][k]/Ndbn[i][k]+Nhln_std[i][k]/Nhln[i][k])
           
           Text_dir_dbn_dbn_corr_hf = '%s/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Doublon_doublon_correlation_functions'%(Text_dir_main_corr_hf,N,u[i],dtau,mu,dtau,L[k])
           Text_dir_den_dbn_corr_hf = '%s/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Density_doublon_correlation_functions'%(Text_dir_main_corr_hf,N,u[i],dtau,mu,dtau,L[k])
           Text_dir_dbn_dbn_conn_corr_hf = '%s/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Doublon_doublon_connected_correlation_functions'%(Text_dir_main_corr_hf,N,u[i],dtau,mu,dtau,L[k])
           Text_dir_den_dbn_conn_corr_hf = '%s/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Density_doublon_connected_correlation_functions'%(Text_dir_main_corr_hf,N,u[i],dtau,mu,dtau,L[k])

           filename_dbn_dbn_corr = '%s/Equal_time_Doublon_doublon_correlation_real_space_neighbor_averaged_normal_avg_N_%s_U_%s_mu_%s_dtau_%s_L_%s.dat'%(Text_dir_dbn_dbn_corr_hf,N,u[i],mu,dtau,L[k])
           Dbn_Dbn_corr_hf[i,k,:],Dbn_Dbn_corr_hf_std[i,k,:] = np.loadtxt(filename_dbn_dbn_corr,unpack = 'True', usecols = [1,2])

           filename_den_dbn_corr = '%s/Equal_time_Density_Doublon_correlation_real_space_neighbor_averaged_normal_avg_N_%s_U_%s_mu_%s_dtau_%s_L_%s.dat'%(Text_dir_den_dbn_corr_hf,N,u[i],mu,dtau,L[k])
           Den_Dbn_corr_hf[i,k,:],Den_Dbn_corr_hf_std[i,k,:] = np.loadtxt(filename_den_dbn_corr,unpack = 'True', usecols = [1,2])

           filename_dbn_dbn_conn_corr = '%s/Equal_time_Doublon_doublon_connected_correlation_real_space_neighbor_averaged_normal_avg_N_%s_U_%s_mu_%s_dtau_%s_L_%s.dat'%(Text_dir_dbn_dbn_conn_corr_hf,N,u[i],mu,dtau,L[k])
           Dbn_Dbn_conn_corr_hf[i,k,:],Dbn_Dbn_conn_corr_hf_std[i,k,:] = np.loadtxt(filename_dbn_dbn_conn_corr,unpack = 'True', usecols = [1,2])

           filename_den_dbn_conn_corr = '%s/Equal_time_Density_Doublon_connected_correlation_real_space_neighbor_averaged_normal_avg_N_%s_U_%s_mu_%s_dtau_%s_L_%s.dat'%(Text_dir_den_dbn_conn_corr_hf,N,u[i],mu,dtau,L[k])
           Den_Dbn_conn_corr_hf[i,k,:],Den_Dbn_conn_corr_hf_std[i,k,:] = np.loadtxt(filename_den_dbn_conn_corr,unpack = 'True', usecols = [1,2])

           Hln_Dbn_corr_hf[i,k,:] = np.add(np.subtract(Dbn_Dbn_corr_hf[i,k,:],Den_Dbn_corr_hf[i,k,:]),Ndbn[i][k]*np.ones(len(Den_Dbn_corr_hf[i,k,:])))
           Hln_Dbn_corr_hf_std[i,k,:] = np.sqrt(np.add(np.add(np.power(Dbn_Dbn_corr_hf_std[i,k,:],2),np.power(Den_Dbn_corr_hf_std[i,k,:],2)),Ndbn_std[i][k]*Ndbn_std[i][k]*Ndbn[i][k]*np.ones(len(Den_Dbn_conn_corr_hf[i,k,:]))))
           
           Hln_Dbn_conn_corr_hf[i,k,:] = np.subtract(Dbn_Dbn_conn_corr_hf[i,k,:],Den_Dbn_conn_corr_hf[i,k,:])
           Hln_Dbn_conn_corr_hf_std[i,k,:] = np.sqrt(np.add(np.power(Dbn_Dbn_conn_corr_hf_std[i,k,:],2),np.power(Den_Dbn_conn_corr_hf_std[i,k,:],2)))

           for r in range(len(r_2_unq)):
               
               Nrm_Hln_Dbn_conn_corr_hf[i,k,r] = (1/(Ndbn[i][k]*Nhln[i][k]))*Hln_Dbn_conn_corr_hf[i,k,r]
               Nrm_Hln_Dbn_conn_corr_hf_std[i,k,r] = Nrm_Hln_Dbn_conn_corr_hf[i,k,r]*(Hln_Dbn_conn_corr_hf_std[i,k,r]/Hln_Dbn_conn_corr_hf[i,k,r]+Hl_db_std[i][k]/Hl_db[i][k])
               
 
   Hln_Dbn_corr_hf_n0 = np.zeros((len(u),len(L)))
   Hln_Dbn_corr_hf_n1 = np.zeros((len(u),len(L)))
   Hln_Dbn_corr_hf_n2 = np.zeros((len(u),len(L)))
   Hln_Dbn_corr_hf_n3 = np.zeros((len(u),len(L)))

   Hln_Dbn_corr_hf_n0_std = np.zeros((len(u),len(L)))
   Hln_Dbn_corr_hf_n1_std = np.zeros((len(u),len(L)))
   Hln_Dbn_corr_hf_n2_std = np.zeros((len(u),len(L)))
   Hln_Dbn_corr_hf_n3_std = np.zeros((len(u),len(L)))

   Hln_Dbn_conn_corr_hf_n0 = np.zeros((len(u),len(L)))
   Hln_Dbn_conn_corr_hf_n1 = np.zeros((len(u),len(L)))
   Hln_Dbn_conn_corr_hf_n2 = np.zeros((len(u),len(L)))
   Hln_Dbn_conn_corr_hf_n3 = np.zeros((len(u),len(L)))
    
   Hln_Dbn_conn_corr_hf_n0_std = np.zeros((len(u),len(L)))
   Hln_Dbn_conn_corr_hf_n1_std = np.zeros((len(u),len(L)))
   Hln_Dbn_conn_corr_hf_n2_std = np.zeros((len(u),len(L)))
   Hln_Dbn_conn_corr_hf_n3_std = np.zeros((len(u),len(L)))

   Nrm_Hln_Dbn_conn_corr_hf_n0 = np.zeros((len(u),len(L)))
   Nrm_Hln_Dbn_conn_corr_hf_n1 = np.zeros((len(u),len(L)))
   Nrm_Hln_Dbn_conn_corr_hf_n2 = np.zeros((len(u),len(L)))
   Nrm_Hln_Dbn_conn_corr_hf_n3 = np.zeros((len(u),len(L)))
    
   Nrm_Hln_Dbn_conn_corr_hf_n0_std = np.zeros((len(u),len(L)))
   Nrm_Hln_Dbn_conn_corr_hf_n1_std = np.zeros((len(u),len(L)))
   Nrm_Hln_Dbn_conn_corr_hf_n2_std = np.zeros((len(u),len(L)))
   Nrm_Hln_Dbn_conn_corr_hf_n3_std = np.zeros((len(u),len(L)))


   for i in range(len(u)):
       for k in range(len(L)):

           Hln_Dbn_corr_hf_n0[i][k] = Hln_Dbn_corr_hf[i][k][0]
           Hln_Dbn_corr_hf_n1[i][k] = Hln_Dbn_corr_hf[i][k][1]
           Hln_Dbn_corr_hf_n2[i][k] = Hln_Dbn_corr_hf[i][k][2]
           Hln_Dbn_corr_hf_n3[i][k] = Hln_Dbn_corr_hf[i][k][3]

           Hln_Dbn_corr_hf_n0_std[i][k] = Hln_Dbn_corr_hf_std[i][k][0]
           Hln_Dbn_corr_hf_n1_std[i][k] = Hln_Dbn_corr_hf_std[i][k][1]
           Hln_Dbn_corr_hf_n2_std[i][k] = Hln_Dbn_corr_hf_std[i][k][2]
           Hln_Dbn_corr_hf_n3_std[i][k] = Hln_Dbn_corr_hf_std[i][k][3]

           Hln_Dbn_conn_corr_hf_n0[i][k] = Hln_Dbn_conn_corr_hf[i][k][0]
           Hln_Dbn_conn_corr_hf_n1[i][k] = Hln_Dbn_conn_corr_hf[i][k][1]
           Hln_Dbn_conn_corr_hf_n2[i][k] = Hln_Dbn_conn_corr_hf[i][k][2]
           Hln_Dbn_conn_corr_hf_n3[i][k] = Hln_Dbn_conn_corr_hf[i][k][3]
           
           Hln_Dbn_conn_corr_hf_n0_std[i][k] = Hln_Dbn_conn_corr_hf_std[i][k][0]
           Hln_Dbn_conn_corr_hf_n1_std[i][k] = Hln_Dbn_conn_corr_hf_std[i][k][1]
           Hln_Dbn_conn_corr_hf_n2_std[i][k] = Hln_Dbn_conn_corr_hf_std[i][k][2]
           Hln_Dbn_conn_corr_hf_n3_std[i][k] = Hln_Dbn_conn_corr_hf_std[i][k][3]

           Nrm_Hln_Dbn_conn_corr_hf_n0[i][k] = Nrm_Hln_Dbn_conn_corr_hf[i][k][0]
           Nrm_Hln_Dbn_conn_corr_hf_n1[i][k] = Nrm_Hln_Dbn_conn_corr_hf[i][k][1]
           Nrm_Hln_Dbn_conn_corr_hf_n2[i][k] = Nrm_Hln_Dbn_conn_corr_hf[i][k][2]
           Nrm_Hln_Dbn_conn_corr_hf_n3[i][k] = Nrm_Hln_Dbn_conn_corr_hf[i][k][3]
           
           Nrm_Hln_Dbn_conn_corr_hf_n0_std[i][k] = Nrm_Hln_Dbn_conn_corr_hf_std[i][k][0]
           Nrm_Hln_Dbn_conn_corr_hf_n1_std[i][k] = Nrm_Hln_Dbn_conn_corr_hf_std[i][k][1]
           Nrm_Hln_Dbn_conn_corr_hf_n2_std[i][k] = Nrm_Hln_Dbn_conn_corr_hf_std[i][k][2]
           Nrm_Hln_Dbn_conn_corr_hf_n3_std[i][k] = Nrm_Hln_Dbn_conn_corr_hf_std[i][k][3]
           
           
   Pl_title_n0 = r"$\langle h_i d_j \rangle_{C}$ |i-j|=0.0, $N=%sx%s, n = 1$"%(N,N)
   Pl_title_n1 = r"$\langle h_i d_j \rangle_{C}$ |i-j|=1.0, $N=%sx%s, n = 1$"%(N,N)
   Pl_title_n2 = r"$\langle h_i d_j \rangle_{C}$ |i-j|=1.414, $N=%sx%s, n = 1$"%(N,N)
   Pl_title_n3 = r"$\langle h_i d_j \rangle_{C}$ |i-j|=2.0, $N=%sx%s, n = 1$"%(N,N)

   Pl_y_lab = r"$\langle h_i d_j \rangle_{C}$"
   Pl_x_lab = r"$U$"

   Pl_name_n0 = "%s/Holon_doublon_correlation_onsite_normal_averaged_half_filling_N_%s_dtau_%s.png"%(Graph_dir_hf,N,dtau)
   Pl_name_conn_n0 = "%s/Holon_doublon_correlation_connected_onsite_normal_averaged_half_filling_N_%s_dtau_%s.png"%(Graph_dir_hf,N,dtau)
   Pl_name_nrm_conn_n0 = "%s/Normalized_Holon_doublon_correlation_connected_onsite_normal_averaged_half_filling_N_%s_dtau_%s.png"%(Graph_dir_hf,N,dtau)

   Pl_name_dbn = "%s/Doublon_number_normal_averaged_half_filling_N_%s_dtau_%s.png"%(Graph_dir_hf,N,dtau)
   
   Pl_name_n1 = "%s/Holon_doublon_correlation_1st_nearest_neighbor_averaged_normal_averaged_half_filling_N_%s_dtau_%s.png"%(Graph_dir_hf,N,dtau)
   Pl_name_conn_n1 = "%s/Holon_doublon_correlation_connected_1st_nearest_neighbor_averaged_normal_averaged_half_filling_N_%s_dtau_%s.png"%(Graph_dir_hf,N,dtau)
   Pl_name_nrm_conn_n1 = "%s/Normalized_Holon_doublon_correlation_connected_1st_nearest_neighbor_averaged_normal_averaged_half_filling_N_%s_dtau_%s.png"%(Graph_dir_hf,N,dtau)
   Pl_name_conn_n2 = "%s/Holon_doublon_correlation_connected_2nd_nearest_neighbor_averaged_normal_averaged_half_filling_N_%s_dtau_%s.png"%(Graph_dir_hf,N,dtau)
   
   #plot_func_multiple_beta_hf(N,u,mu,dtau,L,Pl_title_n1,Pl_x_lab,Pl_y_lab,U_val,Hln_Dbn_corr_hf_n0,Hln_Dbn_corr_hf_n1_std,Pl_name_n0)
   #plot_func_multiple_beta_hf(N,u,mu,dtau,L,Pl_title_n2,Pl_x_lab,Pl_y_lab,U_val,Hln_Dbn_conn_corr_hf_n0,Hln_Dbn_conn_corr_hf_n1_std,Pl_name_conn_n0)
   #plot_func_multiple_beta_hf(N,u,mu,dtau,L,Pl_title_n3,Pl_x_lab,Pl_y_lab,U_val,Nrm_Hln_Dbn_conn_corr_hf_n0,Nrm_Hln_Dbn_conn_corr_hf_n1_std,Pl_name_nrm_conn_n0)
   
   #plot_func_multiple_beta_hf(N,u,mu,dtau,L,Pl_title_n1,Pl_x_lab,Pl_y_lab,U_val,Hln_Dbn_corr_hf_n1,Hln_Dbn_corr_hf_n1_std,Pl_name_n1)

   #plot_func_multiple_beta_hf(N,u,mu,dtau,L,Pl_title_n2,Pl_x_lab,Pl_y_lab,U_val,Hln_Dbn_conn_corr_hf_n1,Hln_Dbn_conn_corr_hf_n1_std,Pl_name_conn_n1)
   plot_func_multiple_beta_hf(N,u,mu,dtau,L,Pl_title_n2,Pl_x_lab,Pl_y_lab,U_val,Hln_Dbn_conn_corr_hf_n2,Hln_Dbn_conn_corr_hf_n2_std,Pl_name_conn_n2)
   #plot_func_multiple_beta_hf(N,u,mu,dtau,L,Pl_title_n3,Pl_x_lab,Pl_y_lab,U_val,Nrm_Hln_Dbn_conn_corr_hf_n1,Nrm_Hln_Dbn_conn_corr_hf_n1_std,Pl_name_nrm_conn_n1)
   
   #plot_func_multiple_beta_hf(N,u,mu,dtau,L,Pl_title_n3,Pl_x_lab,Pl_y_lab,U_val,Ndbn,Ndbn_std,Pl_name_dbn)
   


In [119]:
def main():

    N = "10"
    Dtau = "0.05"
    U = ["1.00","1.50","2.00","2.50","3.00","3.50","4.00","4.50","5.00","5.50","6.00","6.50","7.00","7.50","8.00","8.50","9.00","9.50","10.00","10.50","11.00","11.50","12.00"]
    Mu ="0.00"
    Trot = ["20","30","40","50","60"]
    
    Text_dir_main_corr_hf = '/Users/roy.369/Documents/Cold_atom_correlators/FHM_Legacy_data/FHM_Roy_data/Text_files/Text_files_N_%s_real_space_correlations_half_filling'%N
    Text_dir_main_eqm_hf = '/Users/roy.369/Documents/Cold_atom_correlators/FHM_Legacy_data/FHM_Roy_data/Text_files/Text_files_N_%s_half_filling'%N
    
    Graph_dir_hf = '/Users/roy.369/Documents/Cold_atom_correlators/FHM_Legacy_data/FHM_Roy_data/Graphs/Graphs_N_%s_half_filling/Graphs_N_%s_dtau_%s_real_space_correlations_normal_averaged/Doublon_holon_correlations'%(N,N,Dtau)
    if not os.path.exists(Graph_dir_hf):
        os.makedirs(Graph_dir_hf)

        
    equal_time_holon_doublon_correlation_function_half_filling(Text_dir_main_corr_hf,Text_dir_main_eqm_hf,Graph_dir_hf,N,U,Mu,Dtau,Trot)


In [120]:
main()

[-0.00014676  0.04199404]
